In [1]:
import xarray as xr 
import matplotlib.pyplot as plt
import cmocean as cmo 
import pandas as pd 
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
# python -m ipykernel install --user --name schimpy --display-name Schimpy

# Start date 
start_date = '2024-08-13'
end_date = pd.to_datetime(start_date) + pd.Timedelta(seconds=(10*8640))
dates = pd.date_range(start_date, end_date, freq='10s')
dates = dates[0:-1]

label = "trying_to_converge"


import matplotlib.dates as mdates

def format_date_ax(ax):
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))
    date_format = mdates.DateFormatter('%m/%d %H:%M')
    ax.xaxis.set_major_formatter(date_format)



In [2]:
# open datasets
hydro = xr.open_dataset('run_hydro/HYDRO.nc', decode_timedelta=False)
truth = xr.open_dataset('forward_phyto/phyto_fake_truth_august_13.nc', decode_timedelta=False)
z = hydro["z"]

NITER = 90

guess = {} 
for i in range(1, NITER):
    guess[i] = xr.open_dataset(f'forward_phyto/forward_{i}_august_13.nc', decode_timedelta=False)

# Read in adjoint 
adjoint = {}
for i in range(1, NITER):
    adjoint[i] = xr.open_dataset(f'backward_lambda/adjoint_{i}_august_13.nc', decode_timedelta=False)



KeyboardInterrupt: 

In [ ]:
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.38888889e-06
 -1.38888889e-06 -1.38888889e-06]
-1.2418957407978164e-06 -1.8733077887135327e-06

In [17]:

import matplotlib.colors as colors


# divnorm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax )


vmin, vmax = (-0.027000000000000003, 0.0)
norm = colors.Normalize(vmin=vmin, vmax=vmax)


for i in range(1, NITER, 2):
    fig = plt.figure(figsize=(10, 3))
    ax = fig.add_subplot(111)
    c = ax.pcolormesh(dates, -z, adjoint[i]['lambda'].values.T , cmap=cmo.cm.haline, norm=norm)#, vmin=1.5e-7, vmax=1.5e-7)
    plt.colorbar(c, shrink=0.8)
    format_date_ax(ax)
    plt.title('Lagrangian multiplier (NITER=%d)' % i)
    fig.savefig("gif1/%d.png" % i, dpi=300)
    plt.close() 



In [31]:

cmax = 1.8733077887135327e-06 * 86400
divnorm = colors.TwoSlopeNorm(vmin=-cmax, vcenter=0, vmax=cmax )




i = 10

for i in range(1, NITER, 2):
    fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
    c = axs[0].pcolormesh(dates, -z, truth.gamma.values.T*86400, cmap=cmo.cm.balance, norm=divnorm)#, vmin=1.5e-7, vmax=1.5e-7)
    plt.colorbar(c, shrink=0.8, label="Growth rate (1/d)")
    format_date_ax(axs[0])
    axs[0].set_title('"True gamma"')
    plt.title('First adjusted gamma (using adjoint)')


    c = axs[1].pcolormesh(dates, -z,  adjoint[i].gamma.values.T*86400, cmap=cmo.cm.balance, norm=divnorm)#, vmin=1.5e-7, vmax=1.5e-7)
    format_date_ax(axs[0])
    plt.colorbar(c, shrink=0.8, label="Growth rate (1/d)")
    axs[1].set_title('Guess #%d'% i)
    axs[0].set_ylabel('Depth (m)')
    axs[1].set_ylabel('Depth (m)')

    fig.savefig("gif2/%d.png" % i)
    plt.close()

# for i in range(1, NITER, 2):
#     fig = plt.figure(figsize=(10, 3))
#     ax = fig.add_subplot(111)
#     c = ax.pcolormesh(dates, -z, adjoint[i]['lambda'].values.T , cmap=cmo.cm.haline, norm=norm)#, vmin=1.5e-7, vmax=1.5e-7)
#     plt.colorbar(c, shrink=0.8)
#     format_date_ax(ax)
#     plt.title('Lagrangian multiplier (NITER=%d)' % i)
#     fig.savefig("gif1/%d.png" % i, dpi=300)
#     plt.close() 



In [4]:


import imageio
images = []
NITER = 89 
for k in range(1, NITER, 1):
    images.append(imageio.imread("gif1/%d.png" % k))
imageio.mimsave('lagrangian_forward.gif', images, fps=4) #duration=1)

/tmp/ipykernel_762754/303302444.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread("gif1/%d.png" % k))
